# Best Practices

 Build on the Lambda/kinetis example from module 4
 * add tests to the code (unit tests, integration tests), we will use the library ```pytest``` for that
 * ```pipenv install --dev pytest```, it is only needed for development
 * For working in Visual Studio:
     * Select Python Interpreter > View > Command Pallete > "Select: Python Interpreter"
     * Go in terminal to the current folder and type ```pipenv --venv``` to get the python environment of the virtual environment (```<path>/.local/share/virtualenvs/code-AxO42iuz```
     * Copy the name ```<path>/.local/share/virtualenvs/code-AxO42iuz/bin/python``` to the Command Pallete and choose this interpreter
     * We then get a new icon on the left hand side for testing
     * Add the test path ```<path>/.local/share/virtualenvs/code-AxO42iuz/bin/pytest``` through "configure tests"
 * **Unit tests** only test small units/fractions of the code, **Integration tests**, test the entire code

## Unit Tests
* Create a first test
    * In the test folder, we need to creat a file ```__init__.py```, so that python knows that this is a python package
    * create a file ```model_test.py``` and ```model.py```
    * Test it using docker, we need to add the new created ```model.py``` script to te docker file: ```bash docker build -t stream-model-duration:v2 .```
    * 
```
docker run -it --rm \
    -p 8080:8080 \
    -e PREDICTIONS_STREAM_NAME="ride_prediction" \
    -e TEST_RUN="True" \
    -e AWS_DEFAULT_REGION="eu-west-1" \
    stream-model-duration:v2
    ```
    

* To run the tests from the terminal: go to folder ```code```, start virtual env: ```pipenv shell```, then run ```pipenv run pytest tests/``` 